In [8]:
!pip install numpy
!pip install openai


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [9]:
import  openai
import numpy as np

In [10]:
openai_client = openai.OpenAI(api_key="<My Open API key>")

In [11]:
def embedding_model(query, openai_client, model="text-embedding-3-small"):
    # This function calculates and returns an embedding
    embedding = openai_client.embeddings.create(input = [query], model=model).data[0].embedding
    return embedding

In [12]:
def cosine_similarity(vec1, vec2):
    # This function creates cosine similarity using only numpy
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

In [13]:
faq_database = {
    "What is your return policy?": "Our return policy allows customers to return products within 30 days of purchase. Items must be in their original condition and packaging. To initiate a return, visit our return portal and provide your order number and email address.",

    "How do I track my order?": "You can track your order by using the tracking number provided in the shipment confirmation email. Alternatively, you can log in to your account and go to the 'Order History' section to find the tracking link.",

    "What payment methods do you accept?": "We accept all major credit cards (Visa, MasterCard, American Express), PayPal, and Apple Pay. For corporate accounts, we also offer invoicing options. Please contact support for more information on setting up a corporate account.",

    "Can I change or cancel my order after it’s been placed?": "Once an order has been placed, we are unable to modify it directly. However, you can cancel your order within the first hour of placing it through the 'My Orders' section of your account. After that, you’ll need to wait for the order to be delivered and then initiate a return.",

    "What are your shipping options?": "We offer standard, expedited, and overnight shipping. Standard shipping takes 5-7 business days, while expedited shipping takes 2-3 business days. Overnight shipping ensures delivery by the next business day. International shipping options are also available, with delivery times varying by destination.",

    "How do I reset my account password?": "To reset your password, go to the login page and click on 'Forgot Password'. You will receive an email with instructions to reset your password. If you don't see the email, check your spam folder or contact customer support for help.",

    "Do you ship internationally?": "Yes, we ship to select international destinations. International shipping costs and delivery times vary depending on the destination. You can calculate the shipping costs at checkout after providing your address.",

    "What do I do if I receive a damaged or defective product?": "If you receive a damaged or defective product, please contact our customer support within 48 hours of receiving the item. We will provide instructions on how to return the product or arrange for a replacement. Make sure to include photos of the damaged item and packaging for faster processing.",

    "How do I contact customer support?": "You can contact our customer support via email at support@ourcompany.com, or by calling our support line at 1-800-123-4567 during business hours (9 AM to 5 PM, Monday to Friday). We also offer live chat support on our website.",

    "Can I use multiple discount codes on a single order?": "No, our system only allows one discount code per order. However, you can apply store credit or a gift card in addition to a discount code at checkout.",

    "How do I update my shipping address after placing an order?": "If your order has not yet been processed, you can update your shipping address by logging into your account and navigating to the 'My Orders' section. If the order has already been processed or shipped, you will need to contact customer support to discuss possible options.",

    "What should I do if I never received my order?": "If your order has not arrived by the estimated delivery date, first check the tracking information. If the tracking shows the item was delivered but you didn't receive it, contact customer support so we can investigate and resolve the issue.",

    "Do you offer gift wrapping?": "Yes, we offer gift wrapping for an additional fee. You can select the gift wrapping option at checkout, and you can also include a personalized message with the gift.",

    "Can I return a product after 30 days?": "Unfortunately, returns are only accepted within 30 days of the purchase date. If you have extenuating circumstances, please contact customer support to discuss possible exceptions on a case-by-case basis.",

    "What are your business hours?": "Our customer support team is available from 9 AM to 5 PM, Monday through Friday, excluding holidays. Our website is available for orders 24/7.",

    "How do I subscribe to your newsletter?": "To subscribe to our newsletter, scroll to the bottom of our homepage and enter your email in the subscription box. You’ll receive exclusive offers, product updates, and company news directly to your inbox.",

    "What is your warranty policy?": "We offer a one-year warranty on all our products. The warranty covers manufacturing defects but does not cover damage caused by misuse, accidents, or normal wear and tear. To file a warranty claim, contact our customer support team with your order details and a description of the issue.",

    "How can I become a reseller of your products?": "We welcome reseller partnerships! If you're interested in becoming a reseller, please contact our sales team at sales@ourcompany.com with details about your business, and we’ll get back to you with more information.",

    "Do you offer student discounts?": "Yes, we offer a 10% discount for students. To get the discount, sign up with your valid student email, and we will verify your status. After verification, you will receive a unique discount code to use at checkout.",

    "Can I expedite the shipping of my order?": "Yes, you can select expedited or overnight shipping at checkout. Expedited shipping typically takes 2-3 business days, while overnight shipping ensures delivery by the next business day. Please note that expedited shipping costs more than standard shipping."
}

In [14]:
vector_database = {}

for q in faq_database:
    vector_database[q] = embedding_model(q, openai_client)

In [15]:
def find_most_similar_faq(query, faq_vector_db, openai_client):
    query_embedding = embedding_model(query, openai_client)
    best_match = None
    highest_similarity = -1

    for faq_question, faq_embedding in faq_vector_db.items():
        similarity = cosine_similarity(query_embedding, faq_embedding)
        if similarity > highest_similarity:
            best_match = faq_question
            highest_similarity = similarity
    
    return best_match

In [16]:
system_prompt = {
    "role": "system",
    "content": f"""
    You are a helpful E-commerce assistant helping customers with their general questions regarding policies and procedures when buying in our stores.
    Our store sells e-books and courses for IT professionals.
    """
}

In [17]:
def get_answer(question, database):
    return database[question]

In [18]:
def prompt_builder(system_message, context):
    return system_message['content'].format(context)

In [19]:
def rag_chatbot(query, vector_database, faq_database, openai_client):

    # 1. Find the most similar FAQ using cosine similarity
    best_match = find_most_similar_faq(query, vector_database, openai_client)
    best_answer = get_answer(best_match, faq_database)

    # 2. Combine the context (best match) with the query
    augmented_prompt = prompt_builder(system_prompt, best_answer)

    messages = [
        {"role": "system", "content": augmented_prompt},
        {"role": "user", "content": query}
    ]

    # 3. Use OpenAI API to generate a response with context
    response = openai_client.chat.completions.create(
        model = "gpt-4o",
        messages = messages,
        max_tokens = 250
    )

    return response.choices[0].message.content


In [20]:
# Example query
query = "Can I return an item?"
response = rag_chatbot(query, vector_database, faq_database, openai_client)
print(f"Bot: {response}")

Bot: We understand that sometimes a purchase may not meet your expectations. Since we sell e-books and courses, which are digital products, our return policy can vary. Generally, we offer returns or exchanges under certain conditions, such as technical issues with accessing the content or if you accidentally purchased the wrong item.

If you believe you qualify for a return or need assistance, please contact our customer support team. They will be happy to help address your concerns and guide you through the process.
